#### This notebook is part 2 of the quickstart. In part1, we have written pyspark code to perform analysis on uber dataset. In this notebook, we will utilize same dataset and perform same analysis using snowpark python.

In [1]:
#import required libraries
from snowflake.snowpark import Session

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from snowflake.snowpark import functions as F
from snowflake.snowpark import Window
import pandas as pd
import numpy as np

- create a snowflake trail account if you do not have one.
- create a database named "snowpark" and schema named "dev"
- Upload uber dataset to your stage. We will read data from this stage and make some transformations

### Connect to Snowflake and load data

For the snowflake connection we need to set the parameters in the config file. Add in your account details like :user,password, account,database,schema,role and warehouse.
You can set the below values:
- "database":"snowpark"
- "schema": "dev",
- "role": "ACCOUNTADMIN",
- "warehouse": "COMPUTE_WH"

In [2]:
#Snowflake connection info is saved in config.py
from config import SNOWFLAKE_CONN_PROFILE
session = Session.builder.configs(SNOWFLAKE_CONN_PROFILE).create()
session.sql("use role Accountadmin").collect()
session.sql("create database if not exists  {}".format(SNOWFLAKE_CONN_PROFILE['database'])).collect()
session.sql("use database {}".format(SNOWFLAKE_CONN_PROFILE['database'])).collect()
session.sql("create schema if not exists  {}".format(SNOWFLAKE_CONN_PROFILE['schema'])).collect()
session.sql("use schema {}".format(SNOWFLAKE_CONN_PROFILE['schema'])).collect()
session.sql("use warehouse {}".format(SNOWFLAKE_CONN_PROFILE['warehouse']))
print(session.sql('select current_warehouse(), current_database(), current_schema(), current_role()').collect())

[Row(CURRENT_WAREHOUSE()='COMPUTE_WH', CURRENT_DATABASE()='SNOWPARK', CURRENT_SCHEMA()='DEV', CURRENT_ROLE()='ACCOUNTADMIN')]


In the following step, we are retrieving the Uber data from the 'Uber' table that was created during the data migration process and loading it into a dataframe.

In [3]:
df = session.sql('select * from uber')
df.schema

StructType([StructField('DATE', StringType(), nullable=True), StructField('"Time (Local)"', LongType(), nullable=True), StructField('EYEBALLS', LongType(), nullable=True), StructField('ZEROES', LongType(), nullable=True), StructField('"Completed Trips"', LongType(), nullable=True), StructField('REQUESTS', LongType(), nullable=True), StructField('"Unique Drivers"', LongType(), nullable=True)])

In [4]:
df.show()

----------------------------------------------------------------------------------------------------------
|"DATE"     |"Time (Local)"  |"EYEBALLS"  |"ZEROES"  |"Completed Trips"  |"REQUESTS"  |"Unique Drivers"  |
----------------------------------------------------------------------------------------------------------
|10-Sep-12  |7               |5           |0         |2                  |2           |9                 |
|10-Sep-12  |8               |6           |0         |2                  |2           |14                |
|10-Sep-12  |9               |8           |3         |0                  |0           |14                |
|10-Sep-12  |10              |9           |2         |0                  |1           |14                |
|10-Sep-12  |11              |11          |1         |4                  |4           |11                |
|10-Sep-12  |12              |12          |0         |2                  |2           |11                |
|10-Sep-12  |13              |9      

We can use Pandas on top of snowpark for example, we can convert the 'Date' column to datetime format. The function 'data_cleaning' creates an additional column called 'DATE_TIME'. Since the 'Date' column is initially a string type, it is necessary to convert it to datetime format for further analysis. The 'DATE_TIME' column is created by combining the 'Date' and 'Time' columns. The data_cleaning function takes an argument,dataframe that contains the uber data.

In [5]:
def data_cleaning(dataframe):
    df1=dataframe.with_column_renamed(col("Time (Local)"), "Time")
    df2=df1.with_column_renamed(col("Completed Trips"), "Completed_Trips")
    df3=df2.with_column_renamed(col("Unique Drivers"), "Unique_Drivers")
    # df3.show()
    uber_pd = df3.toPandas()
    uber_pd['DATE_TIME'] = uber_pd['DATE'].astype('str')+' '+uber_pd['TIME'].astype('str')
    uber_pd['DATE_TIME']=pd.to_datetime(uber_pd['DATE_TIME'], format='%d-%b-%y %H')
    uber_pd['DATE']=pd.to_datetime(uber_pd['DATE'], format='%d-%b-%y')
    #uber_df.head()
     
    return uber_pd


In [6]:
uber=data_cleaning(df)
uber

,DATE,TIME,EYEBALLS,ZEROES,COMPLETED_TRIPS,REQUESTS,UNIQUE_DRIVERS,DATE_TIME
0,2012-09-10,7,5,0,2,2,9,2012-09-10 07:00:00
1,2012-09-10,8,6,0,2,2,14,2012-09-10 08:00:00
2,2012-09-10,9,8,3,0,0,14,2012-09-10 09:00:00
3,2012-09-10,10,9,2,0,1,14,2012-09-10 10:00:00
4,2012-09-10,11,11,1,4,4,11,2012-09-10 11:00:00
...,...,...,...,...,...,...,...,...
331,2012-09-24,2,3,3,0,2,0,2012-09-24 02:00:00
332,2012-09-24,3,3,3,0,1,0,2012-09-24 03:00:00
333,2012-09-24,4,1,1,0,0,0,2012-09-24 04:00:00
334,2012-09-24,5,4,2,1,1,3,2012-09-24 05:00:00


In [7]:
uber.dtypes

DATE               datetime64[ns]
TIME                         int8
EYEBALLS                     int8
ZEROES                       int8
COMPLETED_TRIPS              int8
REQUESTS                     int8
UNIQUE_DRIVERS               int8
DATE_TIME          datetime64[ns]
dtype: object

In [8]:
type(uber)

pandas.core.frame.DataFrame

In [9]:
uber['DATE']=uber['DATE'].dt.tz_localize('UTC')
uber['DATE_TIME']=uber['DATE_TIME'].dt.tz_localize('UTC')

In [10]:
uber_df=session.create_dataframe(uber)
uber_df.show()

----------------------------------------------------------------------------------------------------------------------------------
|"DATE"               |"TIME"  |"EYEBALLS"  |"ZEROES"  |"COMPLETED_TRIPS"  |"REQUESTS"  |"UNIQUE_DRIVERS"  |"DATE_TIME"          |
----------------------------------------------------------------------------------------------------------------------------------
|2012-09-10 00:00:00  |7       |5           |0         |2                  |2           |9                 |2012-09-10 07:00:00  |
|2012-09-10 00:00:00  |8       |6           |0         |2                  |2           |14                |2012-09-10 08:00:00  |
|2012-09-10 00:00:00  |9       |8           |3         |0                  |0           |14                |2012-09-10 09:00:00  |
|2012-09-10 00:00:00  |10      |9           |2         |0                  |1           |14                |2012-09-10 10:00:00  |
|2012-09-10 00:00:00  |11      |11          |1         |4                  |4      

In [11]:
uber_df1=uber_df.select(to_date(col('DATE')).alias('DATE'),"TIME","EYEBALLS","ZEROES","COMPLETED_TRIPS","REQUESTS","UNIQUE_DRIVERS","DATE_TIME")
uber_df1.show()

-------------------------------------------------------------------------------------------------------------------------
|"DATE"      |"TIME"  |"EYEBALLS"  |"ZEROES"  |"COMPLETED_TRIPS"  |"REQUESTS"  |"UNIQUE_DRIVERS"  |"DATE_TIME"          |
-------------------------------------------------------------------------------------------------------------------------
|2012-09-10  |7       |5           |0         |2                  |2           |9                 |2012-09-10 07:00:00  |
|2012-09-10  |8       |6           |0         |2                  |2           |14                |2012-09-10 08:00:00  |
|2012-09-10  |9       |8           |3         |0                  |0           |14                |2012-09-10 09:00:00  |
|2012-09-10  |10      |9           |2         |0                  |1           |14                |2012-09-10 10:00:00  |
|2012-09-10  |11      |11          |1         |4                  |4           |11                |2012-09-10 11:00:00  |
|2012-09-10  |12      |1

### 1. Which date had the most completed trips during the two-week period?

In [12]:
def max_trips_date(dataframe):
    trips_per_date = dataframe.groupBy('DATE').agg(sum('COMPLETED_TRIPS').alias('total_completed_trips')).sort("total_completed_trips", ascending=False)
    trips_per_date=trips_per_date.select('Date').limit(1)
    return trips_per_date

In [13]:
max_trips=max_trips_date(uber_df1)
max_trips.show()

--------------
|"DATE"      |
--------------
|2012-09-22  |
--------------



### 2. What was the highest number of completed trips within a 24-hour period?

In [14]:
def highest_completed_trips(dataframe):
    # Group the data by 24-hour window and sum the completed trips
    completed_trips_24hrs = dataframe.groupBy(F.date_trunc('day', 'DATE_TIME'),floor(hour('DATE_TIME')/24)) \
                                   .agg(F.sum("completed_Trips").alias("Total_Completed_Trips"))

    # Get the highest number of completed trips within a 24-hour period
    highest_completed_trips_in_24_hours =completed_trips_24hrs.select(max('Total_Completed_Trips').alias('Total_Completed_Trips_in_24hrs')).first()[0]
    return highest_completed_trips_in_24_hours

In [15]:
highest_trips = highest_completed_trips(uber_df1)
highest_trips

248

### 3. Which hour of the day had the most requests during the two-week period?

In [16]:
def most_requests_per_hour(dataframe):
    df_hour = dataframe.groupBy("TIME").agg(sum('REQUESTS').alias('Total_Requests')).sort("Total_Requests", ascending=False)
    most_req_hr = df_hour.select('TIME').first()[0]
    return most_req_hr

In [17]:
most_requests = most_requests_per_hour(uber_df1)
most_requests

23

### 4. What percentages of all zeroes during the two-week period occurred on weekend (Saturday and Sunday)

#### In snowpark, satuarday day of week is 6 and Sunday day of week is 0

In [18]:
def percentage_zeroes(dataframe):
    # count number of zeros that occurred on weekends
    weekend_df = dataframe.withColumn("is_weekend", when(dayofweek(col("DATE_TIME")).isin([0,6]), 1).otherwise(0))
    weekend_zeroes = weekend_df.filter(col("is_weekend") == 1).agg(sum('ZEROES').alias('WEEKEND_ZEROES')).select('WEEKEND_ZEROES').first()["WEEKEND_ZEROES"]

    # total number of zeros
    total_zeroes = dataframe.agg(sum("ZEROES").alias('TOTAL_ZEROES')).collect()[0]['TOTAL_ZEROES']
    per_zero=(weekend_zeroes/total_zeroes)*100
    return per_zero


In [19]:
per_zeroes = percentage_zeroes(uber_df1)
print(per_zeroes)

47.93561931420574


### 5. What is the weighted average ratio of completed trips per driver during the two-week period? Tip: “Weighted average” means your answer should account for the total trip volume in each hour to determine the most accurate number in the whole period.

In [20]:
from snowflake.snowpark.functions import floor
def weighted_avg_ratio(dataframe):
    weighted_avg_ratio = dataframe.withColumn("trips_per_driver", when(dataframe["Unique_Drivers"]==0, 1).otherwise(dataframe["Completed_Trips"] / dataframe["Unique_Drivers"])) \
                 .groupBy("DATE", "Time") \
                 .agg(avg("trips_per_driver").alias("avg_completed_per_driver"), sum("Completed_trips").alias("total_completed_trips")) \
                 .withColumn("weighted_ratio", col("avg_completed_per_driver") * col("total_completed_trips")) \
                 .agg(sum("weighted_ratio") / sum("total_completed_trips"))
    war=weighted_avg_ratio.with_column_renamed(col("DIVIDE(SUM(WEIGHTED_RATIO), SUM(TOTAL_COMPLETED_TRIPS))"), "Weighted_average")\
                          .select(trunc(col("Weighted_average"), lit(3)).alias('Weighted_average'))
    return war


In [21]:
weighted_avg = weighted_avg_ratio(uber_df1)
weighted_avg.show()

----------------------
|"WEIGHTED_AVERAGE"  |
----------------------
|0.828               |
----------------------



### 6. In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours over the two-week period in terms of unique requests? A new shift starts every 8 hours. Assume that a driver will work the same shift each day.

In [22]:
uber_df.groupBy('Time').agg(countDistinct('Requests').alias('Total_Requests')).show()

-----------------------------
|"TIME"  |"TOTAL_REQUESTS"  |
-----------------------------
|7       |5                 |
|8       |6                 |
|9       |6                 |
|10      |4                 |
|11      |8                 |
|12      |8                 |
|13      |8                 |
|15      |8                 |
|17      |10                |
|18      |9                 |
-----------------------------



In [23]:
def busiest_hours(dataframe):
    requests_per_hour = dataframe.groupBy('Time').agg(countDistinct('Requests').alias('Total_Requests'))
    window_8hr = Window.orderBy(col('Total_Requests').desc()).rowsBetween(0,7)

    busiest_8_hrs = requests_per_hour.select('*', sum('Total_Requests').over(window_8hr).alias("sum_8_hrs"))\
                                 .orderBy(col("sum_8_hrs").desc()).limit(1)
    return busiest_8_hrs

In [24]:
busiest_8_hours = busiest_hours(uber_df1)
busiest_8_hours.show()

-------------------------------------------
|"TIME"  |"TOTAL_REQUESTS"  |"SUM_8_HRS"  |
-------------------------------------------
|20      |12                |80           |
-------------------------------------------



### 7. In which 72-hour period is the ratio of Zeroes to Eyeballs the highest?

In [25]:
def zeroes_to_eyeballs_ratio(dataframe):
    # Group the data by 72-hour periods and calculate the ratio of zeroes to eyeballs for each period
    period_ratios = (dataframe.groupBy((hour(col("DATE_TIME")) / (72*3600)).cast("int"))\
                            .agg(sum("Zeroes").alias("zeroes"), sum("Eyeballs").alias("eyeballs"))\
                            .withColumn("ratio", col("zeroes") / col("eyeballs")))

    # Find the period with the highest ratio
    highest_ratio_period = period_ratios.orderBy(col("ratio").desc()).limit(1)
    hrp=highest_ratio_period.with_column_renamed(col("CAST(DIVIDE(HOUR(DATE_TIME), LITERAL()))"), "PERIOD")
    return hrp

In [26]:
zeroes_to_eyeballs = zeroes_to_eyeballs_ratio(uber_df1)
zeroes_to_eyeballs.show()

-----------------------------------------------
|"PERIOD"  |"ZEROES"  |"EYEBALLS"  |"RATIO"   |
-----------------------------------------------
|0         |1429      |6687        |0.213698  |
-----------------------------------------------



### 8. If you could add 5 drivers to any single hour of every day during the two-week period, which hour should you add them to? Hint: Consider both rider eyeballs and driver supply when choosing

In [27]:
def requests_per_driver(dataframe):
    requests_per_driver = (dataframe.groupBy('Time')\
                         .agg((sum('Requests') / countDistinct('Unique_Drivers')).alias('requests_per_driver')))

    requests_per_driver= requests_per_driver.sort('requests_per_driver', ascending=False).limit(1)
    return requests_per_driver

In [28]:
requests_driver = requests_per_driver(uber_df1)
requests_driver.show()

----------------------------------
|"TIME"  |"REQUESTS_PER_DRIVER"  |
----------------------------------
|2       |20.000000              |
----------------------------------



### 9. Looking at the data from all two weeks, which time might make the most sense to consider a true “end day” instead of midnight? (i.e when are supply and demand at both their natural minimums)

In [29]:
def true_end_day(dataframe):
    true_end_day = dataframe.groupBy('Time')\
       .agg(avg('Completed_Trips').alias('avg_requests'), avg('Unique_Drivers').alias('avg_unique_drivers'))\
       .orderBy('avg_requests', 'avg_unique_drivers').limit(1)
    return true_end_day


In [30]:
true_endday = true_end_day(uber_df1)
true_endday.show()

--------------------------------------------------
|"TIME"  |"AVG_REQUESTS"  |"AVG_UNIQUE_DRIVERS"  |
--------------------------------------------------
|4       |0.142857        |0.642857              |
--------------------------------------------------

